In [1]:
import math
import numpy as np
import pandas as pd
import sklearn
import scipy
import networkx as nx
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import coo_matrix, vstack

In [2]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

In [3]:
device_type = {'user_id':str, 'device_id_0':str, 'device_id_1':str, 'device_id_2':str, 'ip':str, 'reach_time':str, 'reach_time_day':str}
cookie_type = {'user_id':str, 'cookieid':str, 'ip':str, 'data_time':str}


In [4]:
%%time
device_df = pd.read_csv('../alidata/data/ijcai_device_encode_training.csv', dtype=device_type)

CPU times: user 1min 7s, sys: 8.95 s, total: 1min 16s
Wall time: 1min 37s


In [5]:
%%time
cookie_df = pd.read_csv('../alidata/data/ijcai_cookie_encode_training.csv', dtype=cookie_type)

CPU times: user 1min 37s, sys: 11 s, total: 1min 48s
Wall time: 2min 22s


In [6]:
len(device_df)

29158167

In [7]:
device_df.head()

,user_id,device_id_0,device_id_1,device_id_2,ip,search_keyword,auction_id,shop_id,geohash6,geohash7,geohash8,reach_time,os
0,000157f178ac0da34f8922aba3be58b1,e658d7862413ed37d305eba287a973ad,9200911503e4c0ca7559d8a8e6c0455b,NaN,3593efe532ee076e931de631dca691ec.0ec04218402b4...,37cdee4aa95e4169c273ecf7e57027cc 0e52d4f14bb62...,NaN,NaN,NaN,NaN,NaN,20170507080341,Android
1,000157f178ac0da34f8922aba3be58b1,e658d7862413ed37d305eba287a973ad,9200911503e4c0ca7559d8a8e6c0455b,NaN,3593efe532ee076e931de631dca691ec.0ec04218402b4...,37cdee4aa95e4169c273ecf7e57027cc 0e52d4f14bb62...,c7abe6d4d2337642b883638767cd13df,ca1eaf152ac91a78577ce7b278a88cff,NaN,NaN,NaN,20170507081459,Android
2,000157f178ac0da34f8922aba3be58b1,e658d7862413ed37d305eba287a973ad,9200911503e4c0ca7559d8a8e6c0455b,NaN,3593efe532ee076e931de631dca691ec.0ec04218402b4...,37cdee4aa95e4169c273ecf7e57027cc 0e52d4f14bb62...,NaN,NaN,NaN,NaN,NaN,20170507092521,Android
3,000157f178ac0da34f8922aba3be58b1,e658d7862413ed37d305eba287a973ad,9200911503e4c0ca7559d8a8e6c0455b,NaN,3593efe532ee076e931de631dca691ec.0ec04218402b4...,37cdee4aa95e4169c273ecf7e57027cc 0e52d4f14bb62...,NaN,NaN,NaN,NaN,NaN,20170507092521,Android
4,000157f178ac0da34f8922aba3be58b1,e658d7862413ed37d305eba287a973ad,9200911503e4c0ca7559d8a8e6c0455b,NaN,3593efe532ee076e931de631dca691ec.0ec04218402b4...,5458553fa1150bd9295e7a64163eea6e,NaN,NaN,NaN,NaN,NaN,20170501165227,Android


In [8]:
len(cookie_df)

15066709

In [9]:
cookie_df.head()

,user_id,data_time,cookieid,url,url_domain1,url_domain2,search_keyword,auction_id,shop_id,ip,title
0,002724ba2ed196b37d62ea478a139b8d,20170507111217,ea96698e0ac16cc5aaba9fb0b2c8f34e,5b16ed3fe6456f7d73f9ded59ab6bcb0/c28bf7bee965d...,3e717b288d611cf2f5dc6dd94c59e8ea,415b5e22510385dc83143d314005632a,NaN,NaN,NaN,d41ea1af79f76adeb7764b3fd63fc68d.0ec04218402b4...,ebbdbaeaa341bb8e7a7114ecf8e09772 9058079915a01...
1,002724ba2ed196b37d62ea478a139b8d,20170507234802,ea96698e0ac16cc5aaba9fb0b2c8f34e,5b16ed3fe6456f7d73f9ded59ab6bcb0/c28bf7bee965d...,3e717b288d611cf2f5dc6dd94c59e8ea,415b5e22510385dc83143d314005632a,NaN,NaN,NaN,63a3999f249475f530693d57bad83b8d.bd00614e2a194...,ebbdbaeaa341bb8e7a7114ecf8e09772 9058079915a01...
2,002724ba2ed196b37d62ea478a139b8d,20170507123950,ea96698e0ac16cc5aaba9fb0b2c8f34e,5b16ed3fe6456f7d73f9ded59ab6bcb0/c28bf7bee965d...,3e717b288d611cf2f5dc6dd94c59e8ea,415b5e22510385dc83143d314005632a,NaN,NaN,NaN,63a3999f249475f530693d57bad83b8d.bd00614e2a194...,ebbdbaeaa341bb8e7a7114ecf8e09772 9058079915a01...
3,002724ba2ed196b37d62ea478a139b8d,20170507111418,ea96698e0ac16cc5aaba9fb0b2c8f34e,f9601526cbe74adb91394bf363f49fbb/c28bf7bee965d...,3e717b288d611cf2f5dc6dd94c59e8ea,a01662de147eeee349d62a5c25cf999b,NaN,NaN,NaN,d41ea1af79f76adeb7764b3fd63fc68d.0ec04218402b4...,5a2b982eb589374e8b1e70a5045fc0de cd3ecb69d0756...
4,002724ba2ed196b37d62ea478a139b8d,20170507111304,ea96698e0ac16cc5aaba9fb0b2c8f34e,94b50e99f4739068f9c732eafd8739e8/c28bf7bee965d...,3e717b288d611cf2f5dc6dd94c59e8ea,786cef0c6a9de12b11ef0366986d1974,NaN,NaN,NaN,d41ea1af79f76adeb7764b3fd63fc68d.0ec04218402b4...,2b4189224280ea828cca9435c5231db0 3cf8a8d74f215...


In [ ]:
device_id_connection_df = device_df[['device_id_0', 'device_id_1']]
device_id_connection_df = device_id_connection_df.drop_duplicates(subset=['device_id_0', 'device_id_1'])
len(device_id_connection_df)

In [9]:
device_id_connection_df = device_df[['device_id_0', 'device_id_1']]

In [10]:
device_id_connection_df = device_id_connection_df.drop_duplicates(subset=['device_id_0', 'device_id_1'])

In [11]:
len(device_id_connection_df)

286737

In [14]:
%%time
device_g=nx.from_pandas_edgelist(device_id_connection_df, 'device_id_0', 'device_id_1')
device_g.remove_nodes_from([np.nan])

CPU times: user 1.86 s, sys: 152 ms, total: 2.01 s
Wall time: 2.01 s


In [16]:
for node in device_g.nodes(data=True):
    if 'type' in node[1].keys():
        pass
    else:
        node[1]['type'] = 'device'

In [15]:
(device_g.number_of_nodes(), device_g.number_of_edges())

(544916, 260993)

In [17]:
connected_device_id = nx.connected_components(device_g)

node_number_list = list()
node_list = list()

for index, connected_device_item in enumerate(sorted(connected_device_id, key=len, reverse=True)):
    node_number_list.append(len(connected_device_item))
    node_list.append(list(connected_device_item))
    
total_connected_group_number = len(node_number_list)

In [18]:
total_connected_group_number

283923

In [19]:
node_number_list[0:10]

[35, 23, 14, 12, 9, 7, 7, 7, 6, 6]

In [29]:
device_mapping_one_to_original = list()
device_mapping_original_to_one = list()


for node_group in node_list:
    
    first_id = node_group[0]
    
    for each_id in node_group:
        device_mapping_one_to_original.append([first_id, each_id])
        device_mapping_original_to_one.append([each_id, first_id])
    

In [31]:
len(device_mapping_original_to_one)

544916

In [32]:
device_mapping_df = pd.DataFrame(device_mapping_original_to_one)
device_mapping_df.columns = ['source_device_id', 'device_id']

In [33]:
len(device_mapping_df)

544916

In [34]:
device_mapping_df.head()

,source_device_id,device_id
0,441c7d15479435c6a9a2acacf06deae5,441c7d15479435c6a9a2acacf06deae5
1,d9e5772f19d8bc3ba586ba0a339721b5,441c7d15479435c6a9a2acacf06deae5
2,f13c2984f77ed2c4e88e934e8949a4fd,441c7d15479435c6a9a2acacf06deae5
3,07d2d565aa3b84b6a26b252e4c323191,441c7d15479435c6a9a2acacf06deae5
4,a0c5fd347776292655d6e17c9b9f575b,441c7d15479435c6a9a2acacf06deae5


In [35]:
device_mapping_df.to_csv('../alidata/data/device_mapping_train.csv', index=False, header=True)

In [36]:
device_mapping_df.set_index('source_device_id', inplace=True)

In [37]:
device_df.set_index('device_id_0', inplace=True)

In [45]:
%%time
device_df_with_mapping = device_df.join(device_mapping_df)

CPU times: user 15.3 s, sys: 4.23 s, total: 19.6 s
Wall time: 19.6 s


In [46]:
device_df_with_mapping.head()

,user_id,device_id_1,device_id_2,ip,search_keyword,auction_id,shop_id,geohash6,geohash7,geohash8,reach_time,os,device_id
00007b3b618f2b616854f4d74cfdde7c,fb3b4f20bc4d5b8346d1fe20dd89ba78,dc8cdfc596d369f232c2d431c4958e61,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,44b8b396a80d775523db7a6c2154efda 1f533b3a58e3b...,NaN,NaN,NaN,NaN,NaN,20170506110511,Android,dc8cdfc596d369f232c2d431c4958e61
00007b3b618f2b616854f4d74cfdde7c,fb3b4f20bc4d5b8346d1fe20dd89ba78,dc8cdfc596d369f232c2d431c4958e61,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,44b8b396a80d775523db7a6c2154efda e579211e16a3e...,NaN,NaN,NaN,NaN,NaN,20170506104958,Android,dc8cdfc596d369f232c2d431c4958e61
00007b3b618f2b616854f4d74cfdde7c,fb3b4f20bc4d5b8346d1fe20dd89ba78,dc8cdfc596d369f232c2d431c4958e61,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,44b8b396a80d775523db7a6c2154efda e579211e16a3e...,50c5f83051f4fd8c5ed5821aae68a806,fdbf986db5aef38b25871fea2c90621c,NaN,NaN,NaN,20170506104926,Android,dc8cdfc596d369f232c2d431c4958e61
00007b3b618f2b616854f4d74cfdde7c,fb3b4f20bc4d5b8346d1fe20dd89ba78,dc8cdfc596d369f232c2d431c4958e61,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,44b8b396a80d775523db7a6c2154efda e579211e16a3e...,NaN,NaN,NaN,NaN,NaN,20170506105031,Android,dc8cdfc596d369f232c2d431c4958e61
00007b3b618f2b616854f4d74cfdde7c,fb3b4f20bc4d5b8346d1fe20dd89ba78,dc8cdfc596d369f232c2d431c4958e61,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,44b8b396a80d775523db7a6c2154efda 1f533b3a58e3b...,NaN,NaN,NaN,NaN,NaN,20170506105324,Android,dc8cdfc596d369f232c2d431c4958e61


In [47]:
device_df_with_mapping.reset_index(inplace=True)

In [48]:
device_df_with_mapping.columns

Index(['index', 'user_id', 'device_id_1', 'device_id_2', 'ip',
       'search_keyword', 'auction_id', 'shop_id', 'geohash6', 'geohash7',
       'geohash8', 'reach_time', 'os', 'device_id'],
      dtype='object')

,index,user_id,device_id_1,device_id_2,ip,search_keyword,auction_id,shop_id,geohash6,geohash7,geohash8,reach_time,os,device_id
0,00007b3b618f2b616854f4d74cfdde7c,fb3b4f20bc4d5b8346d1fe20dd89ba78,dc8cdfc596d369f232c2d431c4958e61,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,44b8b396a80d775523db7a6c2154efda 1f533b3a58e3b...,NaN,NaN,NaN,NaN,NaN,20170506110511,Android,dc8cdfc596d369f232c2d431c4958e61
1,00007b3b618f2b616854f4d74cfdde7c,fb3b4f20bc4d5b8346d1fe20dd89ba78,dc8cdfc596d369f232c2d431c4958e61,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,44b8b396a80d775523db7a6c2154efda e579211e16a3e...,NaN,NaN,NaN,NaN,NaN,20170506104958,Android,dc8cdfc596d369f232c2d431c4958e61
2,00007b3b618f2b616854f4d74cfdde7c,fb3b4f20bc4d5b8346d1fe20dd89ba78,dc8cdfc596d369f232c2d431c4958e61,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,44b8b396a80d775523db7a6c2154efda e579211e16a3e...,50c5f83051f4fd8c5ed5821aae68a806,fdbf986db5aef38b25871fea2c90621c,NaN,NaN,NaN,20170506104926,Android,dc8cdfc596d369f232c2d431c4958e61
3,00007b3b618f2b616854f4d74cfdde7c,fb3b4f20bc4d5b8346d1fe20dd89ba78,dc8cdfc596d369f232c2d431c4958e61,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,44b8b396a80d775523db7a6c2154efda e579211e16a3e...,NaN,NaN,NaN,NaN,NaN,20170506105031,Android,dc8cdfc596d369f232c2d431c4958e61
4,00007b3b618f2b616854f4d74cfdde7c,fb3b4f20bc4d5b8346d1fe20dd89ba78,dc8cdfc596d369f232c2d431c4958e61,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,44b8b396a80d775523db7a6c2154efda 1f533b3a58e3b...,NaN,NaN,NaN,NaN,NaN,20170506105324,Android,dc8cdfc596d369f232c2d431c4958e61


In [50]:
device_df_with_mapping.columns = ['device_id_0', 'user_id', 'device_id_1', 'device_id_2', 'ip', 'search_keyword',
       'auction_id', 'shop_id', 'geohash6', 'geohash7', 'geohash8',
       'reach_time', 'os', 'device_id_from_0']

In [51]:
device_df_with_mapping.set_index('device_id_1', inplace=True)

In [52]:
%%time
device_df_with_mapping = device_df_with_mapping.join(device_mapping_df)

CPU times: user 11.2 s, sys: 2.88 s, total: 14.1 s
Wall time: 14.1 s


In [53]:
device_df_with_mapping.head()

,device_id_0,user_id,device_id_2,ip,search_keyword,auction_id,shop_id,geohash6,geohash7,geohash8,reach_time,os,device_id_from_0,device_id
00001088566af342b421f9d13a5f9d17,f63a7ac8a39bd8365fd3cbb3ca5d7ece,7ca92be942981d88c88cb2111ccaa283,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,3a0419f4545277349c54772eaf0eb9f3,NaN,a917325b9ec0eef767f2547517d51f27,NaN,NaN,NaN,20170503010056,Android,00001088566af342b421f9d13a5f9d17,00001088566af342b421f9d13a5f9d17
00001088566af342b421f9d13a5f9d17,f63a7ac8a39bd8365fd3cbb3ca5d7ece,7ca92be942981d88c88cb2111ccaa283,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,3a0419f4545277349c54772eaf0eb9f3,NaN,NaN,NaN,NaN,NaN,20170503010506,Android,00001088566af342b421f9d13a5f9d17,00001088566af342b421f9d13a5f9d17
00001088566af342b421f9d13a5f9d17,f63a7ac8a39bd8365fd3cbb3ca5d7ece,7ca92be942981d88c88cb2111ccaa283,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,05a2338992b14cad5a6fb558640a33e2 3b446e081b380...,NaN,NaN,NaN,NaN,NaN,20170503010717,Android,00001088566af342b421f9d13a5f9d17,00001088566af342b421f9d13a5f9d17
00001088566af342b421f9d13a5f9d17,f63a7ac8a39bd8365fd3cbb3ca5d7ece,7ca92be942981d88c88cb2111ccaa283,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,05a2338992b14cad5a6fb558640a33e2 15de2152867a1...,NaN,NaN,NaN,NaN,NaN,20170503011105,Android,00001088566af342b421f9d13a5f9d17,00001088566af342b421f9d13a5f9d17
00001088566af342b421f9d13a5f9d17,f63a7ac8a39bd8365fd3cbb3ca5d7ece,7ca92be942981d88c88cb2111ccaa283,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,05a2338992b14cad5a6fb558640a33e2 3b446e081b380...,8ab7b5701b923e1babad577e2cebc824,d3158d724850a3773931bb03fb06b797,NaN,NaN,NaN,20170503010611,Android,00001088566af342b421f9d13a5f9d17,00001088566af342b421f9d13a5f9d17


In [54]:
device_df_with_mapping.reset_index(inplace=True)

In [55]:
device_df_with_mapping.head()

,index,device_id_0,user_id,device_id_2,ip,search_keyword,auction_id,shop_id,geohash6,geohash7,geohash8,reach_time,os,device_id_from_0,device_id
0,00001088566af342b421f9d13a5f9d17,f63a7ac8a39bd8365fd3cbb3ca5d7ece,7ca92be942981d88c88cb2111ccaa283,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,3a0419f4545277349c54772eaf0eb9f3,NaN,a917325b9ec0eef767f2547517d51f27,NaN,NaN,NaN,20170503010056,Android,00001088566af342b421f9d13a5f9d17,00001088566af342b421f9d13a5f9d17
1,00001088566af342b421f9d13a5f9d17,f63a7ac8a39bd8365fd3cbb3ca5d7ece,7ca92be942981d88c88cb2111ccaa283,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,3a0419f4545277349c54772eaf0eb9f3,NaN,NaN,NaN,NaN,NaN,20170503010506,Android,00001088566af342b421f9d13a5f9d17,00001088566af342b421f9d13a5f9d17
2,00001088566af342b421f9d13a5f9d17,f63a7ac8a39bd8365fd3cbb3ca5d7ece,7ca92be942981d88c88cb2111ccaa283,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,05a2338992b14cad5a6fb558640a33e2 3b446e081b380...,NaN,NaN,NaN,NaN,NaN,20170503010717,Android,00001088566af342b421f9d13a5f9d17,00001088566af342b421f9d13a5f9d17
3,00001088566af342b421f9d13a5f9d17,f63a7ac8a39bd8365fd3cbb3ca5d7ece,7ca92be942981d88c88cb2111ccaa283,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,05a2338992b14cad5a6fb558640a33e2 15de2152867a1...,NaN,NaN,NaN,NaN,NaN,20170503011105,Android,00001088566af342b421f9d13a5f9d17,00001088566af342b421f9d13a5f9d17
4,00001088566af342b421f9d13a5f9d17,f63a7ac8a39bd8365fd3cbb3ca5d7ece,7ca92be942981d88c88cb2111ccaa283,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,05a2338992b14cad5a6fb558640a33e2 3b446e081b380...,8ab7b5701b923e1babad577e2cebc824,d3158d724850a3773931bb03fb06b797,NaN,NaN,NaN,20170503010611,Android,00001088566af342b421f9d13a5f9d17,00001088566af342b421f9d13a5f9d17


In [56]:
device_df_with_mapping.columns

Index(['index', 'device_id_0', 'user_id', 'device_id_2', 'ip',
       'search_keyword', 'auction_id', 'shop_id', 'geohash6', 'geohash7',
       'geohash8', 'reach_time', 'os', 'device_id_from_0', 'device_id'],
      dtype='object')

In [57]:
device_df_with_mapping.columns=['device_id_1', 'device_id_0', 'user_id', 'device_id_2', 'ip',
       'search_keyword', 'auction_id', 'shop_id', 'geohash6', 'geohash7',
       'geohash8', 'reach_time', 'os', 'device_id_from_0', 'device_id_from_1']

In [63]:
device_df_with_mapping['device_id_from_0'].fillna(device_df_with_mapping['device_id_from_1'], inplace=True)

In [64]:
%%time
device_id_0_is_null = device_df_with_mapping[pd.isnull(device_df_with_mapping['device_id_from_0'])]

CPU times: user 1 s, sys: 12.1 ms, total: 1.02 s
Wall time: 1.01 s


In [66]:
device_df_with_mapping = device_df_with_mapping[pd.notnull(device_df_with_mapping['device_id_from_0'])]

In [67]:
len(device_df_with_mapping)

29158141

In [65]:
len(device_id_0_is_null)

26

In [68]:
device_df_with_mapping.head()

,device_id_1,device_id_0,user_id,device_id_2,ip,search_keyword,auction_id,shop_id,geohash6,geohash7,geohash8,reach_time,os,device_id_from_0,device_id_from_1
0,00001088566af342b421f9d13a5f9d17,f63a7ac8a39bd8365fd3cbb3ca5d7ece,7ca92be942981d88c88cb2111ccaa283,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,3a0419f4545277349c54772eaf0eb9f3,NaN,a917325b9ec0eef767f2547517d51f27,NaN,NaN,NaN,20170503010056,Android,00001088566af342b421f9d13a5f9d17,00001088566af342b421f9d13a5f9d17
1,00001088566af342b421f9d13a5f9d17,f63a7ac8a39bd8365fd3cbb3ca5d7ece,7ca92be942981d88c88cb2111ccaa283,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,3a0419f4545277349c54772eaf0eb9f3,NaN,NaN,NaN,NaN,NaN,20170503010506,Android,00001088566af342b421f9d13a5f9d17,00001088566af342b421f9d13a5f9d17
2,00001088566af342b421f9d13a5f9d17,f63a7ac8a39bd8365fd3cbb3ca5d7ece,7ca92be942981d88c88cb2111ccaa283,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,05a2338992b14cad5a6fb558640a33e2 3b446e081b380...,NaN,NaN,NaN,NaN,NaN,20170503010717,Android,00001088566af342b421f9d13a5f9d17,00001088566af342b421f9d13a5f9d17
3,00001088566af342b421f9d13a5f9d17,f63a7ac8a39bd8365fd3cbb3ca5d7ece,7ca92be942981d88c88cb2111ccaa283,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,05a2338992b14cad5a6fb558640a33e2 15de2152867a1...,NaN,NaN,NaN,NaN,NaN,20170503011105,Android,00001088566af342b421f9d13a5f9d17,00001088566af342b421f9d13a5f9d17
4,00001088566af342b421f9d13a5f9d17,f63a7ac8a39bd8365fd3cbb3ca5d7ece,7ca92be942981d88c88cb2111ccaa283,NaN,63a3999f249475f530693d57bad83b8d.fb98177bac9c7...,05a2338992b14cad5a6fb558640a33e2 3b446e081b380...,8ab7b5701b923e1babad577e2cebc824,d3158d724850a3773931bb03fb06b797,NaN,NaN,NaN,20170503010611,Android,00001088566af342b421f9d13a5f9d17,00001088566af342b421f9d13a5f9d17


In [69]:
device_df_with_mapping.columns

Index(['device_id_1', 'device_id_0', 'user_id', 'device_id_2', 'ip',
       'search_keyword', 'auction_id', 'shop_id', 'geohash6', 'geohash7',
       'geohash8', 'reach_time', 'os', 'device_id_from_0', 'device_id_from_1'],
      dtype='object')

In [70]:
device_df_with_mapping.columns = ['device_id_1', 'device_id_0', 'user_id', 'device_id_2', 'ip',
       'search_keyword', 'auction_id', 'shop_id', 'geohash6', 'geohash7',
       'geohash8', 'reach_time', 'os', 'union_device_id', 'device_id_from_1']

In [71]:
device_df_with_mapping.to_csv('../alidata/data/ijcai_device_encode_training_union_id.csv', header=True, index=False)

In [254]:
def count_with_time(target_df, key_name, target_column_name, time_column_name, length=8):
    traget_count_df = target_df[[key_name, target_column_name, time_column_name]].copy()
    traget_count_df['reach_time_day'] = traget_count_df[time_column_name].map(lambda x:x[0:length])
    traget_count_df['target_with_time'] = traget_count_df[target_column_name] + '_' + traget_count_df['reach_time_day']
    
    target_count_with_time_df = traget_count_df[[key_name, target_column_name, 'target_with_time']].\
                                drop_duplicates(subset=[key_name, target_column_name])
    
    target_count_with_time_gb = target_count_with_time_df.groupby([key_name, target_column_name]).count()
    
    target_count_with_time_gb.reset_index(inplace=True)
    
    return target_count_with_time_gb
    
    

In [265]:
def count_with_split(target_df, key_name, target_column_name, drop_duplicates=True):
    
    if (drop_duplicates):
        traget_count_df = target_df[[key_name, target_column_name]].drop_duplicates(subset=[key_name, target_column_name])
    else:
        traget_count_df = target_df[[key_name, target_column_name]]
        
    target_split = traget_count_df[target_column_name].str.split(' ', expand=True)
    target_split = target_split.stack()
    target_split = target_split.reset_index(level=1, drop=True)
    target_split = target_split.rename('target_split')        
    
    target_count_df_join = traget_count_df.join(target_split)
    
    target_count_df_join.drop([target_column_name], axis=1, inplace=True)
    
    target_count_df_join.reset_index(inplace=True)
    target_count_df_join['value'] = 1
    target_count_gb = target_count_df_join.groupby([key_name, 'target_split']).count()
    target_count_gb.reset_index(inplace=True)
    
    return target_count_gb


In [258]:
def count_column(target_df, key_name, target_column_name):

    target_count_df = target_df[[key_name, target_column_name]].copy()
    target_count_df = target_count_df[pd.notnull(target_count_df[target_column_name])]
    target_count_df['value'] = 1
    target_count_gb = target_count_df.groupby([key_name, target_column_name]).count()
    target_count_gb.reset_index(inplace=True)
    
    return target_count_gb


In [259]:
cookie_df.head()

,user_id,data_time,cookieid,url,url_domain1,url_domain2,search_keyword,auction_id,shop_id,ip,title
0,002724ba2ed196b37d62ea478a139b8d,20170507111217,ea96698e0ac16cc5aaba9fb0b2c8f34e,5b16ed3fe6456f7d73f9ded59ab6bcb0/c28bf7bee965d...,3e717b288d611cf2f5dc6dd94c59e8ea,415b5e22510385dc83143d314005632a,NaN,NaN,NaN,d41ea1af79f76adeb7764b3fd63fc68d.0ec04218402b4...,ebbdbaeaa341bb8e7a7114ecf8e09772 9058079915a01...
1,002724ba2ed196b37d62ea478a139b8d,20170507234802,ea96698e0ac16cc5aaba9fb0b2c8f34e,5b16ed3fe6456f7d73f9ded59ab6bcb0/c28bf7bee965d...,3e717b288d611cf2f5dc6dd94c59e8ea,415b5e22510385dc83143d314005632a,NaN,NaN,NaN,63a3999f249475f530693d57bad83b8d.bd00614e2a194...,ebbdbaeaa341bb8e7a7114ecf8e09772 9058079915a01...
2,002724ba2ed196b37d62ea478a139b8d,20170507123950,ea96698e0ac16cc5aaba9fb0b2c8f34e,5b16ed3fe6456f7d73f9ded59ab6bcb0/c28bf7bee965d...,3e717b288d611cf2f5dc6dd94c59e8ea,415b5e22510385dc83143d314005632a,NaN,NaN,NaN,63a3999f249475f530693d57bad83b8d.bd00614e2a194...,ebbdbaeaa341bb8e7a7114ecf8e09772 9058079915a01...
3,002724ba2ed196b37d62ea478a139b8d,20170507111418,ea96698e0ac16cc5aaba9fb0b2c8f34e,f9601526cbe74adb91394bf363f49fbb/c28bf7bee965d...,3e717b288d611cf2f5dc6dd94c59e8ea,a01662de147eeee349d62a5c25cf999b,NaN,NaN,NaN,d41ea1af79f76adeb7764b3fd63fc68d.0ec04218402b4...,5a2b982eb589374e8b1e70a5045fc0de cd3ecb69d0756...
4,002724ba2ed196b37d62ea478a139b8d,20170507111304,ea96698e0ac16cc5aaba9fb0b2c8f34e,94b50e99f4739068f9c732eafd8739e8/c28bf7bee965d...,3e717b288d611cf2f5dc6dd94c59e8ea,786cef0c6a9de12b11ef0366986d1974,NaN,NaN,NaN,d41ea1af79f76adeb7764b3fd63fc68d.0ec04218402b4...,2b4189224280ea828cca9435c5231db0 3cf8a8d74f215...


In [255]:
%%time
cookie_ip_count_result_df = count_with_time(cookie_df, 'cookieid', 'ip', 'data_time', 8)

CPU times: user 19.1 s, sys: 4.92 s, total: 24 s
Wall time: 24 s


In [260]:
%%time
cookie_auction_count_result_df = count_column(cookie_df, 'cookieid', 'auction_id')

In [261]:
%%time
cookie_shop_count_result_df = count_column(cookie_df, 'cookieid', 'shop_id')

CPU times: user 3.53 s, sys: 412 ms, total: 3.94 s
Wall time: 3.94 s


In [266]:
%%time
cookie_keyword_count_result_df = count_with_split(cookie_df, 'cookieid', 'search_keyword')

CPU times: user 1min 40s, sys: 4.89 s, total: 1min 45s
Wall time: 1min 45s


In [271]:
%%time
cookie_ip_count_result_df.to_csv('../alidata/data/sparse_prepare_cookie_ip_count.csv', header=True, index=False)
cookie_auction_count_result_df.to_csv('../alidata/data/sparse_prepare_cookie_auction_count.csv', header=True, index=False)
cookie_shop_count_result_df.to_csv('../alidata/data/sparse_prepare_cookie_shop_count.csv', header=True, index=False)
cookie_keyword_count_result_df.to_csv('../alidata/data/sparse_prepare_cookie_keyword_count.csv', header=True, index=False)


CPU times: user 14.6 s, sys: 452 ms, total: 15 s
Wall time: 15 s


In [276]:
# device_df_with_mapping.head()

In [274]:
%%time

device_ip_count_result_df = count_with_time(device_df_with_mapping, 'union_device_id', 'ip', 'reach_time', 8)
device_auction_count_result_df = count_column(device_df_with_mapping, 'union_device_id', 'auction_id')
device_shop_count_result_df = count_column(device_df_with_mapping, 'union_device_id', 'shop_id')
device_keyword_count_result_df = count_with_split(device_df_with_mapping, 'union_device_id', 'search_keyword')


CPU times: user 3min 18s, sys: 19.4 s, total: 3min 37s
Wall time: 3min 37s


In [275]:
%%time
device_ip_count_result_df.to_csv('../alidata/data/sparse_prepare_device_ip_count.csv', header=True, index=False)
device_keyword_count_result_df.to_csv('../alidata/data/sparse_prepare_device_keyword_count.csv', header=True, index=False)
device_auction_count_result_df.to_csv('../alidata/data/sparse_prepare_device_auction_count.csv', header=True, index=False)
device_shop_count_result_df.to_csv('../alidata/data/sparse_prepare_device_shop_count.csv', header=True, index=False)



CPU times: user 44 s, sys: 1.7 s, total: 45.7 s
Wall time: 48 s


In [214]:
def prepare_for_sparse(target_df, key_name, column1_name, column2_name):
    target_df['accumulate'] = target_df[column1_name] + '_' + target_df[column2_name].astype('str')
    target_gb_to_one = target_df[[key_name,'accumulate']].groupby(key_name)['accumulate'].apply(list)
    
    key_list = list()
    count_list = list()

    for k,v in target_gb_to_one.items():
        key_list.append(k)

        count_dict = dict()

        for count_value in v:
            count_value_split = count_value.split('_')
            count_dict[count_value_split[0]] = int(count_value_split[1])

        count_list.append(count_dict)
        
    return (key_list, count_list)

In [ ]:
%%time
key_list, dict_list = prepare_for_sparse(device_keyword_count_gb, 'union_device_id', 'keyword_split', 'value')


In [ ]:
key_list, dict_list = prepare_for_sparse(device_auction_count_gb, 'union_device_id', 'auction_id', 'value')

In [ ]:
key_list, dict_list = prepare_for_sparse(device_shop_count_gb, 'union_device_id', 'shop_id', 'value')

In [155]:
device_ip_day_count_list[1].keys()

dict_keys(['3593efe532ee076e931de631dca691ec.bd6dfa9ddb96fa517ca10fe0997f3ba4.f3abe6490c42d5ca723536cd173e2dbe.59baa454ecc992a7a3e386ab3083c9da', '95e0c389596951470a30b04ea2f3cdc3.dd8610a3bd76a96a2178cc9cbe0ec016.507a3d4a364ceb54346b7bc0592becc0.32f5c369ba5da48f5f93ebbb7b0ba53d'])

In [156]:
for testing_key in device_ip_day_count_list[1].keys():
    device_ip_day_count_list[0][testing_key] = 2

In [160]:
dict_vectorizer = DictVectorizer(sparse=True)

device_ip_day_features = dict_vectorizer.fit_transform(device_ip_day_count_list)

In [161]:
device_ip_day_features[0]

<1x427735 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [162]:
feature_names = dict_vectorizer.get_feature_names()

In [163]:
len(feature_names)

427735

In [145]:
device_ip_day_features.getrow(1)

<1x427735 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [149]:
cosine_similarity(device_ip_day_features[0:3])

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [94]:
len(device_ip_day_count_gb)

1171116

In [166]:
first_pare = vstack([device_ip_day_features.getrow(0), device_ip_day_features.getrow(1)])

In [167]:
cosine_similarity(first_pare)

array([[1.        , 0.65079137],
       [0.65079137, 1.        ]])

### label pare generating

In [10]:
device_df.reset_index(inplace=True)
cookie_df.reset_index(inplace=True)


In [11]:
def device_cookie_mapping(device_df, cookie_df, mapping_column_name):

    device_id_0_with_target_df = device_df[[mapping_column_name, 'device_id_0']].drop_duplicates(subset=[mapping_column_name, 'device_id_0'])
    device_id_1_with_target_df = device_df[[mapping_column_name, 'device_id_1']].drop_duplicates(subset=[mapping_column_name, 'device_id_1'])

    device_id_0_with_target_df.columns=[mapping_column_name, 'device_id']
    device_id_1_with_target_df.columns=[mapping_column_name, 'device_id']

    device_id_with_target_df = pd.concat([device_id_0_with_target_df, device_id_1_with_target_df])

    device_id_with_target_df = device_id_with_target_df.drop_duplicates(subset=[mapping_column_name, 'device_id'])

    cookieid_with_target_df = cookie_df[[mapping_column_name, 'cookieid']].drop_duplicates(subset=[mapping_column_name, 'cookieid'])


    device_id_with_target_df.set_index(mapping_column_name, inplace=True)
    cookieid_with_target_df.set_index(mapping_column_name, inplace=True)

    device_cookie_target_df = device_id_with_target_df.join(cookieid_with_target_df, how='inner')

    device_cookie_target_df.reset_index(inplace=True)
    
    return device_cookie_target_df


In [12]:
%%time
device_cookie_user_df = device_cookie_mapping(device_df, cookie_df, 'user_id')

CPU times: user 12.4 s, sys: 1.87 s, total: 14.2 s
Wall time: 14.2 s


In [13]:
len(device_cookie_user_df)

1484071

In [14]:
%%time
device_cookie_ip_df = device_cookie_mapping(device_df, cookie_df, 'ip')

CPU times: user 42.2 s, sys: 9.2 s, total: 51.4 s
Wall time: 51.4 s


In [15]:
len(device_cookie_ip_df)

182341289

In [16]:
device_cookie_ip_df.head()

,ip,device_id,cookieid
0,0892f364bdc69014ae45e883fb423e01.5ba07df8130d8...,c6c76386e7e18d45b5d42d84758177bc,1e6d0e9cf07aaccbdc156ab316657d5d
1,0892f364bdc69014ae45e883fb423e01.5ba07df8130d8...,aa8669be11d00784f24328d8e37cfb83,1e6d0e9cf07aaccbdc156ab316657d5d
2,09189b9a21c21a9092e1fd711962ff16.05b2c36ec9f63...,314e8d386af34a49c03a13a72ade6f65,56772cd11904e75e8b5a95c671f5093b
3,09189b9a21c21a9092e1fd711962ff16.05b2c36ec9f63...,314e8d386af34a49c03a13a72ade6f65,50de9ffd18f616282e7c22916d47b7b5
4,09189b9a21c21a9092e1fd711962ff16.05b2c36ec9f63...,ea0d99fb184575c7c36151daaf101642,56772cd11904e75e8b5a95c671f5093b


In [17]:
device_cookie_user_df = device_cookie_user_df.drop_duplicates(subset=['device_id', 'cookieid'])

In [18]:
%%time
device_cookie_user_df.to_csv('../alidata/data/device_cookie_user_training.csv', index=False, header=True)

CPU times: user 3.63 s, sys: 138 ms, total: 3.77 s
Wall time: 3.96 s


In [19]:
device_cookie_ip_df = device_cookie_ip_df.drop_duplicates(subset=['device_id', 'cookieid'])

In [20]:
%%time
device_cookie_ip_df.to_csv('../alidata/data/device_cookie_ip_training.csv', index=False, header=True)

CPU times: user 11min 31s, sys: 27 s, total: 11min 58s
Wall time: 11min 59s


In [ ]:
list(device_mapping_df[['device_id', 'source_device_id']])

In [15]:
cookie_test_df.head()

,user_id,data_time,cookieid,url,url_domain1,url_domain2,search_keyword,auction_id,shop_id,ip,title
0,00107db01724177334168edcde0316de,20170504162141,dfda306f4194f64a2edba8b7850b7916,11d5de9d70262cd0bac56468590ac2e9/c28bf7bee965d...,cc56b7cf02ca811e0f0c4d6c41b2d63d,6eabf3ab911d12248a98ac76bef2bbd9,NaN,NaN,NaN,32f5c369ba5da48f5f93ebbb7b0ba53d.22f5907dcd774...,fe9fbb21f8eb28267794c89bd2195e06 7ec0dcfbef15c...
1,00107db01724177334168edcde0316de,20170504121918,ebfc9734b1d69126367677629fbaf001,355ffb74d915b5af974746bf95270e96/c28bf7bee965d...,3e717b288d611cf2f5dc6dd94c59e8ea,415b5e22510385dc83143d314005632a,NaN,NaN,NaN,1f0ba6bf3b36f1b054f5309567f710b2.1289994d9aa7b...,f40e0ae89f586f7e2688a0768502121d 6abdabe1602d5...
2,00107db01724177334168edcde0316de,20170505221600,410ee5c224fc8de726ff04234353d9e6,5a48c512bb1e9bd5f131ec4ebe005523/c28bf7bee965d...,b2c235606545568a97f3fa3ac7e02860,f82bfe1601e10c6399acfc74fb131999,NaN,79c424ce3b44d11d17668ea56a4988aa,3e236dbf07aaa8edcf15fd11e04aa57b,50beb1e1f9ee8806580ba559ea06dae6.9a66503fbd765...,96c1abd42f500b6c25d0af06cf72e343 b37c13faa585b...
3,00107db01724177334168edcde0316de,20170505221341,410ee5c224fc8de726ff04234353d9e6,5a48c512bb1e9bd5f131ec4ebe005523/c28bf7bee965d...,b2c235606545568a97f3fa3ac7e02860,f82bfe1601e10c6399acfc74fb131999,NaN,79c424ce3b44d11d17668ea56a4988aa,3e236dbf07aaa8edcf15fd11e04aa57b,50beb1e1f9ee8806580ba559ea06dae6.9a66503fbd765...,96c1abd42f500b6c25d0af06cf72e343 b37c13faa585b...
4,00107db01724177334168edcde0316de,20170505221552,410ee5c224fc8de726ff04234353d9e6,5a48c512bb1e9bd5f131ec4ebe005523/c28bf7bee965d...,b2c235606545568a97f3fa3ac7e02860,f82bfe1601e10c6399acfc74fb131999,NaN,1840633b00b8cd8cdc5c707c0d56f861,3e236dbf07aaa8edcf15fd11e04aa57b,50beb1e1f9ee8806580ba559ea06dae6.9a66503fbd765...,45cc045a2363f24668ed6d51e1ef3821 2294255ec6f3a...


In [16]:
cookie_ip_df = cookie_test_df[['user_id', 'cookieid', 'ip', 'data_time']]

In [17]:
cookie_ip_df = cookie_ip_df.drop_duplicates(subset=['user_id', 'cookieid', 'ip', 'data_time'])

In [18]:
cookie_ip_df.head()

,user_id,cookieid,ip,data_time
0,00107db01724177334168edcde0316de,dfda306f4194f64a2edba8b7850b7916,32f5c369ba5da48f5f93ebbb7b0ba53d.22f5907dcd774...,20170504162141
1,00107db01724177334168edcde0316de,ebfc9734b1d69126367677629fbaf001,1f0ba6bf3b36f1b054f5309567f710b2.1289994d9aa7b...,20170504121918
2,00107db01724177334168edcde0316de,410ee5c224fc8de726ff04234353d9e6,50beb1e1f9ee8806580ba559ea06dae6.9a66503fbd765...,20170505221600
3,00107db01724177334168edcde0316de,410ee5c224fc8de726ff04234353d9e6,50beb1e1f9ee8806580ba559ea06dae6.9a66503fbd765...,20170505221341
4,00107db01724177334168edcde0316de,410ee5c224fc8de726ff04234353d9e6,50beb1e1f9ee8806580ba559ea06dae6.9a66503fbd765...,20170505221552


In [19]:
len(cookie_test_df)

9492770

In [20]:
len(cookie_ip_df)

9097513

In [21]:
device_ip_df.to_csv('../alidata/data/clean_device_ip_time.csv', index=False, header=True)

In [22]:
cookie_ip_df.to_csv('../alidata/data/clean_cookie_ip_time.csv', index=False, header=True)